# Permutation Entropy

In [5]:
import numpy as np
import math
from typing import Dict, Tuple, Iterable

def permutation_entropy(
        x: Iterable[float], 
        m: int = 3, 
        tau: int = 1, 
        normalize: bool = True
        ) -> Tuple[float, Dict[Tuple[int, ...], float]]:
    
    x = np.asarray(x)
    n = x.size
    window_span = (m - 1) * tau

    counts: Dict[Tuple[int, ...], int] = {}
    for t in range(n - window_span):
        w = x[t : t + window_span + 1 : tau]
        perm = tuple(np.argsort(w, kind="mergesort"))
        counts[perm] = counts.get(perm, 0) + 1

    total = n - window_span
    p: Dict[Tuple[int, ...], float] = {k: v / total for k, v in counts.items()}

    H = -sum(prob * math.log(prob) for prob in p.values())

    if normalize:
        H /= math.log(math.factorial(m))

    return H, p


In [6]:
# Permutation Entropy (Bandt & Pompe, 2002) — raw, from scratch.
# - Input: 1D time series x
# - Parameters:
#     m   : embedding dimension (pattern length)       [typical 3..7]
#     tau : time delay between embedded samples         [typical 1..5]
# - Output:
#     H   : (normalized) permutation entropy
#     p   : pattern probability dictionary {perm_tuple: prob}
#
# Notes on ties:
# - PE assumes continuous data (ties are rare). For deterministic behavior,
#   we break ties by index order using a STABLE argsort (mergesort).
#   This yields a well-defined permutation for any window without adding noise.

from typing import Dict, Tuple, Iterable
import math
import numpy as np


def permutation_entropy(
    x: Iterable[float],
    m: int = 3,
    tau: int = 1,
    normalize: bool = True,
    base: float = math.e
) -> Tuple[float, Dict[Tuple[int, ...], float]]:
    """
    Compute (normalized) permutation entropy of a 1D sequence.

    Parameters
    ----------
    x : Iterable[float]
        Time series (list/array-like).
    m : int
        Embedding dimension (length of ordinal patterns), m >= 2.
    tau : int
        Time delay between points inside each embedded vector, tau >= 1.
    normalize : bool
        If True, divide by log(base, m!) to get H in [0, 1].
    base : float
        Logarithm base for entropy (e for nats, 2 for bits).

    Returns
    -------
    H : float
        (Normalized) permutation entropy.
    p : Dict[Tuple[int, ...], float]
        Probability of each observed permutation pattern.
        Each key is a tuple of indices (0..m-1) representing the ordinal pattern.
        Example for m=3: (0,1,2) means x[t] < x[t+tau] < x[t+2*tau].
    """
    x = np.asarray(x)
    n = x.size
    if m < 2:
        raise ValueError("m (embedding dimension) must be >= 2")
    if tau < 1:
        raise ValueError("tau (time delay) must be >= 1")
    window_span = (m - 1) * tau
    if n <= window_span:
        raise ValueError(f"Time series too short for m={m}, tau={tau}: need > {(m-1)*tau} offset.")

    # Count occurrences of each ordinal pattern
    counts: Dict[Tuple[int, ...], int] = {}
    # Use stable argsort to break ties by original index order
    for t in range(n - window_span):
        # Extract embedded vector: [x[t], x[t+tau], ..., x[t+(m-1)tau]]
        w = x[t : t + window_span + 1 : tau]
        # Ordinal pattern = argsort indices (stable) of w
        perm = tuple(np.argsort(w, kind="mergesort"))  # stable tie-breaking
        counts[perm] = counts.get(perm, 0) + 1

    total = float(n - window_span)
    # Convert counts to probabilities
    p: Dict[Tuple[int, ...], float] = {k: v / total for k, v in counts.items()}

    # Shannon entropy over observed patterns
    # (Unobserved patterns contribute zero to the sum.)
    H = 0.0
    log = (lambda y: math.log(y, base)) if base != math.e else math.log
    for prob in p.values():
        if prob > 0.0:
            H -= prob * log(prob)

    if normalize:
        # Maximum entropy is log(m!) in the chosen base
        max_H = log(math.factorial(m))
        if max_H > 0:
            H /= max_H

    return H, p


# -------------------------
# Minimal usage examples
if __name__ == "__main__":
    # 1) White noise → PE ~ 1 (high disorder) when normalized
    rng = np.random.default_rng(0)
    x_white = rng.standard_normal(5000)
    H1, p1 = permutation_entropy(x_white, m=5, tau=1, normalize=True, base=math.e)
    print(f"White noise PE (m=5, tau=1): {H1:.3f}  (should be close to 1)")

    # 2) Monotone increasing sequence → PE ~ 0 (one pattern dominates)
    x_inc = np.arange(1000, dtype=float)
    H2, p2 = permutation_entropy(x_inc, m=5, tau=1, normalize=True, base=math.e)
    print(f"Monotone PE (m=5, tau=1): {H2:.3f}  (should be 0)")

    # 3) Periodic sequence → intermediate entropy
    x_sin = np.sin(2 * math.pi * np.arange(2000) / 50.0)
    H3, p3 = permutation_entropy(x_sin, m=5, tau=1, normalize=True, base=math.e)
    print(f"Periodic PE (m=5, tau=1): {H3:.3f}  (intermediate)")

    # Inspect a few pattern probabilities (optional)
    # Sort by probability descending
    top_patterns = sorted(p3.items(), key=lambda kv: kv[1], reverse=True)[:5]
    print("Top 5 patterns (periodic example):")
    for perm, prob in top_patterns:
        print(perm, f"{prob:.4f}")


White noise PE (m=5, tau=1): 0.997  (should be close to 1)
Monotone PE (m=5, tau=1): 0.000  (should be 0)
Periodic PE (m=5, tau=1): 0.270  (intermediate)
Top 5 patterns (periodic example):
(0, 1, 2, 3, 4) 0.4454
(4, 3, 2, 1, 0) 0.4344
(4, 3, 2, 0, 1) 0.0180
(0, 1, 4, 2, 3) 0.0140
(4, 0, 3, 1, 2) 0.0140
